In [49]:
import os
from dotenv import load_dotenv
from IPython.display import JSON
import ccxt
import logging
from math import ceil

logger = logging.getLogger()
logger.setLevel(logging.INFO)

load_dotenv()

True

In [50]:
class Portfolio:
    def __init__(self, ratios, total_buy_volume_krw):
        self.total_buy_volume_krw = total_buy_volume_krw
        
        self.portfolio = {}
        for symbol, ratio in ratios.items():
            self.portfolio[symbol] = {
                'symbol': symbol,
                'ratio' : ratio,
                'buy_volume_krw': ratio * total_buy_volume_krw
            }

    def __iter__(self):
        for val in self.portfolio.values():
            yield val


In [55]:
ratios = {
    'BTC/KRW': 1.666666,
    'ETH/KRW': 0.1,
    'SOL/KRW': 0.1,
    'LINK/KRW': 0.1
}

portfolio = Portfolio(ratios, 50000)

In [56]:
bithumb = ccxt.bithumb(config={
    'apiKey': os.getenv("ACCESS_KEY"),
    'secret': os.getenv("SECRET_KEY"),
    'enableRateLimit': True
})
bithumb.fetch_balance()['KRW']

{'free': 250461.79225, 'used': 0.0, 'total': 250461.79225}

In [57]:
for coin in portfolio:
    amount = coin['buy_volume_krw'] / bithumb.fetch_ticker(coin['symbol'])["close"]
    amount = ceil(amount * 1000000) / 1000000
    logging.info(f'''  Symbol: {coin['symbol']}, Ratio: {coin['ratio']}, Buy_Amount_KRW: {coin['buy_volume_krw']} Buy_Amount_Symbol: { amount }''')
    
    
    resp = bithumb.create_market_buy_order(
        symbol=coin['symbol'],
        amount=amount
    )
    
    logging.info(resp)

INFO:root:  Symbol: BTC/KRW, Ratio: 1.666666, Buy_Amount_KRW: 83333.3 Buy_Amount_Symbol: 0.000548
INFO:root:{'info': {'status': '0000', 'order_id': 'C0101000002341588982'}, 'symbol': 'BTC/KRW', 'type': 'market', 'side': 'buy', 'id': 'C0101000002341588982', 'fees': [], 'clientOrderId': None, 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'price': None, 'amount': None, 'cost': None, 'average': None, 'filled': None, 'remaining': None, 'timeInForce': 'IOC', 'postOnly': None, 'trades': [], 'reduceOnly': None, 'stopPrice': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'status': None, 'fee': None}
INFO:root:  Symbol: ETH/KRW, Ratio: 0.1, Buy_Amount_KRW: 5000.0 Buy_Amount_Symbol: 0.001403
INFO:root:{'info': {'status': '0000', 'order_id': 'C0102000001091021766'}, 'symbol': 'ETH/KRW', 'type': 'market', 'side': 'buy', 'id': 'C0102000001091021766', 'fees': [], 'clientOrderId': None, 'timestamp': None, 'datetime': None, 'l